In [8]:
!pip install wget -q
!pip install openai -q
!pip install yFinance -q

import pandas as pd
import wget
import openai
import os
openai.api_key = "" #replace with api key

#download the ticker database

url = "https://drive.google.com/uc?export=download&id=1BW-te6Qi6UBCobrVeBprW7jffLvNdkcf"
output_file = "tickers.csv"
wget.download(url, out=output_file)
tickers = pd.read_csv("/content/tickers.csv")

In [9]:
import datetime
from datetime import date
import json
import logging
import operator
import sys
import openai
import yfinance as yf
import re

logging.basicConfig(level=logging.WARNING, format="%(asctime)s %(message)s")

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

def get_price(symbol: str, date: str) -> float:
    logger.info(f"Calling get_price with symbol={symbol} and date={date}")

    history = yf.download(symbol, start=date, period="1d", interval="1d", progress=False)

    return history["Close"].iloc[0].item()

def calculate(a: float, b: float, op: str) -> float:
    logger.info(f"Calling calculate with a={a}, b={b}, and op={op}")

    return getattr(operator, op)(a, b)

def ask(user_query: str):
    date_today = date.today()
    get_price_metadata = {
        "name": "get_price",
        "description": "Get closing price of a financial instrument on a given date",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "Ticker symbol of a financial instrument",
                },
                "date": {
                    "type": "string",
                    "description": "Date in the format YYYY-MM-DD",
                },
            },
            "required": ["symbol", "date"],
        },
    }

    calculate_metadata = {
        "name": "calculate",
        "description": "General purpose calculator",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {
                    "type": "number",
                    "description": "First entry",
                },
                "b": {
                    "type": "number",
                    "description": "Second entry",
                },
                "op": {
                    "type": "string",
                    "enum": ["mul", "add", "truediv", "sub"],
                    "description": "Binary operation",
                },
            },
            "required": ["a", "b", "op"],
        },
    }

    system_content = "You are a helpful financial assistant who looks over the performance of stocks. You have to extract the company from the user_request."
    user_content = f"user_request: {user_query}\nCompany_name: "
    m1 = [
        {"role": "system", "content": f"{system_content}"},
        {"role": "user", "content": f"{user_content}"}
    ]
    result = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        max_tokens=20,
        temperature=0.8,
        messages=m1
    )
    response = result["choices"][0]["message"]["content"]
    print("Company Name: " ,response)

    company_name = response.lower()
    lst = []
    tickers = pd.read_csv("tickers.csv")
    for i in range(0, len(tickers)):
      if str(company_name) in (tickers['Company Name'][i]):
        print("Ticker: ",str(tickers['Symbol'][i]))
        lst.append(str(tickers['Symbol'][i]))
        break
    ticker_symbol = lst[0]
    messages = [
        {"role": "user", "content": f"{user_query}\nTicker token: {ticker_symbol} \nToday date: {date_today}"},
        {"role": "system", "content": f"You are a helpful financial investor who overlooks the performance of stocks. Today is {date_today}. Also note that the format of the date is YYYY/MM/DD"}
    ]

    while True:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            temperature=0,
            messages=messages,
            functions=[get_price_metadata, calculate_metadata]
        )
        message = response["choices"][0]["message"]
        messages.append(message)

        if "function_call" not in message:
            break

        # Call custom functions
        function_name = message["function_call"]["name"]
        kwargs = json.loads(message["function_call"]["arguments"])

        if function_name == "get_price":
            output = str(get_price(**kwargs))
        elif function_name == "calculate":
            output = str(calculate(**kwargs))
        else:
            raise ValueError

        messages.append({"role": "function", "name": function_name, "content": output})

    print(messages[-1]["content"])



#Just run this single code block to start querying

In [10]:
ask("How much more is tata price today as compared to 10 years back")

Company Name:  tata
Ticker:  TCS.NS


INFO:__main__:Calling get_price with symbol=TCS.NS and date=2013-07-08
INFO:__main__:Calling get_price with symbol=TCS.NS and date=2023-07-08
INFO:__main__:Calling calculate with a=3329.25, b=753.3250122070312, and op=sub


The price of Tata stock today (2023-07-08) is 3329.25. 

Compared to 10 years ago (2013-07-08), the price was 753.33. 

Therefore, the price of Tata stock today is approximately 2575.92 more than it was 10 years ago.


In [11]:
ask("If I had bought 100 rupees worth of Tata Motors stock in 2012, how much would it be worth today?")

Company Name:  Tata Motors
Ticker:  TATAMOTORS.NS


INFO:__main__:Calling get_price with symbol=TATAMOTORS.NS and date=2012-01-01
INFO:__main__:Calling get_price with symbol=TATAMOTORS.NS and date=2023-07-08
INFO:__main__:Calling calculate with a=618.2000122070312, b=181.99789428710938, and op=truediv
INFO:__main__:Calling calculate with a=100, b=3.396742663581561, and op=mul


If you had bought 100 rupees worth of Tata Motors stock in 2012, it would be worth approximately 339.67 rupees today.


In [12]:
ask("ICICI bank share price today as compared to 10 years ago")

Company Name:  ICICI bank
Ticker:  ICICIBANK.NS


INFO:__main__:Calling get_price with symbol=ICICIBANK.NS and date=2023-07-08
INFO:__main__:Calling get_price with symbol=ICICIBANK.NS and date=2013-07-08
INFO:__main__:Calling calculate with a=946.4000244140625, b=186.89999389648438, and op=sub


The ICICI Bank share price today is 946.40 INR compared to 10 years ago, which was 186.90 INR. Therefore, the share price has increased by 759.50 INR over the past 10 years.


In [13]:
ask("what is apple share price today")

Company Name:  Apple
Ticker:  AAPL


INFO:__main__:Calling get_price with symbol=AAPL and date=2023-07-08


The share price of Apple (AAPL) today is $190.68.


In [14]:
ask("what is Alphabet share price today")

Company Name:  Alphabet
Ticker:  GOOGL


INFO:__main__:Calling get_price with symbol=GOOGL and date=2023-07-08


The share price of Alphabet (GOOGL) today is $119.48.


In [15]:
ask("what is microsoft share price today")

Company Name:  Microsoft
Ticker:  MSFT


INFO:__main__:Calling get_price with symbol=MSFT and date=2023-07-08


The share price of Microsoft (MSFT) today is $337.22.
